## 決定木学習とは
決定木学習は 決定木 と呼ばれる 木構造のグラフ を作る機械学習手法です。機械学習の分野では学習手法も単に「決定木」と呼ばれます。

分類にも回帰にも使え、分類の場合3クラス以上の多値分類が可能です。ここでは基本となる分類のみを扱います。

#### 決定木とは
決定木は、属性 と 値 の組｛属性1：値1，属性2：値2, 属性3：値3,…，属性n：値n｝によって表現されたデータを、条件分岐を繰り返すことであるクラスに割り当てることができる木構造のグラフです。

以下の例は会場の気温という属性の値によって、開催と中止のクラスに割り当てるグラフです。「会場の気温という属性の値は35以上かどうか」という条件分岐1回による決定木による分類が行えます。例えば36度がこの決定木にインプットされれば、中止というアウトプット（判断）ができます。

<img src="https://t.gyazo.com/teams/diveintocode/ca1760b9db2eff08bc82102db1bf7eea.png" width="400">

なお、「属性と値」は機械学習の分野では「特徴量の名前と特徴量の値」のことです。これ以降は単に特徴量という呼びます。

#### 各種用語
もう少し複雑な例で決定木で重要な用語を確認します。特徴量が「雨量」「屋内かどうか」「風の強さ」の3種類で、イベントの開催か中止かを分類する場合で考えてみます。訓練データを学習することで、以下のような決定木が作れます。

<img src="https://t.gyazo.com/teams/diveintocode/c927a798dc2292cc05663301dde78632.png" width="400">

丸で囲われたひとつひとつを ノード と呼びます。ノードには親子関係を考えることができ、例えば(0)のノードは(1)(2)(3)のノードの 親ノード と呼びます。逆に、(1)(2)(3)のノードは(0)のノードの 子ノード と呼びます。

一番上の(0)は 根ノード 、 末端の(1)(4)(5)(7)(8)(9)のような分類結果を表すノードは 葉ノード と呼びます。

条件分岐の矢印は エッジ と呼びます。あるノードから根ノードまでのエッジの数が 深さ です。(3)の深さは1、(6)の深さは2、(9)の深さは3という風になります。この決定木の最大の深さは3です。

これは(0)に対して(1)(2)(3)の3つのノードが分かれている多岐分岐の決定木ですが、機械学習では2つにしか分かれないものが一般的です。学習時の複雑さを減らすためです。

#### どう決定木を作るか
決定木の学習には様々なやり方が存在しますが、その中のある方法についてスクラッチを行いながら見ていきます。

学習方法やハイパーパラメータ、訓練データ次第で作られる決定木は異なってきます。

#### 推定を考える
以下の場合、イベントは開催されるでしょうか。決定木を使って判断してください。

|雨量[mm]|屋内かどうか|風の強さ[m/s]|
|---|---|---|
|2.5|1（屋内）|5|

答えは「開催」です。以下の赤線の順でたどっていきます。

<img src="https://t.gyazo.com/teams/diveintocode/3abf4302fd28b9c58e9c4f86e5878661.png" width="400">

これが決定木による推定の操作になります。

#### 扱える特徴量
決定木は理論上は量的変数だけでなく、カテゴリ変数も扱えます。しかし、scikit-learnの実装では量的変数のみに対応していますので、スクラッチ実装もそのように作成します。上記の例ですと「会場の種類」で「屋内と屋外」ですとカテゴリ変数ですが、「屋内かどうか」で「0と1」と量的変数にすることで扱えるようにしています。

## 決定木スクラッチ
分類のための決定木のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

決定木の学習には何回まで条件分岐を繰り返すかを表す （最大の）深さ というハイパーパラメータが登場しますが、深さ1の実装を必須課題とします。深さが2以上のものはアドバンス課題とします。

学習の仕方には様々な方法がありますが、ここではscikit-learnでも使用されている CART法 をベースとした実装を行います。この方法では学習の複雑さを減らすために、 分岐は2つに分かれるのみ になります。

以下に雛形を用意してあります。このScratchDecesionTreeClassifierDepth1クラスにコードを書き加えていってください。

《雛形》
```
class ScratchDecesionTreeClassifierDepth1():
    """
    深さ1の決定木分類器のスクラッチ実装

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
    def fit(self, X, y):
        """
        決定木分類器を学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
        pass
    def predict(self, X):
        """
        決定木分類器を使いラベルを推定する
        """
        pass
        return
```

#### 分割の条件を学習で求める
学習によって、ノードをどういった条件で分割すると、うまく分けられるかということを求めます。

うまく分けられていることを判定するためにノードに対してジニ不純度と情報利得という値を計算します。

In [1]:
import numpy as np

In [119]:
class ScratchDecesionTreeClassifierDepth1():
    """
    深さ1の決定木分類器のスクラッチ実装

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
        self.list_node = []
        
    def fit(self, X, y):
        """
        決定木分類器を学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
        pass
    
    def predict(self, X):
        """
        決定木分類器を使いラベルを推定する
        """
        pass
        return
    
    def _ginis_diversity_index(self, dict_class_sample):
        """
        ジニ係数を計算する

        Parameters
        ----------
        dict_class_sample : クラスとサンプル数を紐付けた辞書型
        keyがクラス、valueが各クラスのサンプル数
        """
        N = sum(dict_class_sample.values())

        tmp_sum = 0
        for i in dict_class_sample.keys():
            tmp_sum += (dict_class_sample[i]/N)**2
        I_t = 1 - tmp_sum
        
        return I_t
    
    def _information_gain(self, dict_class_sample_left, dict_class_sample_right):
        """
        情報利得を計算する

        Parameters
        ----------
        dict_class_sample_left : クラスとサンプル数を紐付けた辞書型(左ノード)
        dict_class_sample_right : クラスとサンプル数を紐付けた辞書型(右ノード)
        keyがクラス、valueが各クラスのサンプル数
        """
        p_samples = np.array(list(dict_class_sample_left.values())) + np.array(list(dict_class_sample_right.values()))
        dict_class_sample_p = dict(zip(list(dict_class_sample_left.keys()), p_samples)) 

        N_p_all = sum(list(dict_class_sample_p.values()))
        N_left_all = sum(list(dict_class_sample_left.values()))
        N_right_all = sum(list(dict_class_sample_right.values()))

        IG = _ginis_diversity_index(dict_class_sample_p) \
             - (N_left_all/N_p_all)*_ginis_diversity_index(dict_class_sample_left) \
             - (N_right_all/N_p_all)*_ginis_diversity_index(dict_class_sample_right)

        return IG
    

class ScratchDecesionTreeNode():
    """
    ScratchDecesionTreeClassifierDepth1で使用するノード情報
    
    Parameters
    ----------
    node_id : int
      ノードID
      
    Attributes
    ----------
    self.p_node_id : int
      親ノードID
    self.right_node_id : int
      右ノードID(-1の場合は無しを意味する)
    self.left_node_id : int
      左ノードID(-1の場合は無しを意味する)
    self.class_ : int
      葉ノードとなったときに記録されるクラス(-1の場合は葉ノードではないことを意味する)
    """
    def __init__(self, node_id):
        if node_id < 0:
            print("エラー")
            return
        self.node_id = node_id
        self.right_node_id = -1
        self.left_node_id = -1
        self.class_ = -1
        
    def set_right_node_id(self, right_node_id):
        self.right_node_id = right_node_id
        
    def get_right_node_id(self):
        return self.right_node_id
    
    def set_left_node_id(self, left_node_id):
        self.left_node_id = left_node_id
        
    def get_left_node_id(self):
        return self.left_node_id
    
    def set_class(self, class_):
        self.class_ = class_
        
    def get_class(self):
        return self.class_

### 【問題1】不純度を求める関数
ノード の ジニ不純度 を計算する関数を作成してください。ノード $t$ に対するジニ不純度 $I(t)$ は以下の数式で求まります。クラスが混じり合っているほどジニ不純度は高くなります。

$$
I(t)=1-\sum_{i=1}^KP^2(C_i|t)=1-\sum_{i=1}^K(\frac{N_{t,i}}{N_{t,all}})^2
$$

$t$ : ノードのインデックス

$i$ : クラスのインデックス

$K$ : クラスの数

$C_i$ : i番目のクラス

$P(C_i|t)$ :　t番目のノードにおける$C_i$の割合

$N_{t,i}$ : t番目のノードのi番目のクラスに属するサンプル数

$N_{t,all}$ : t番目のノードのサンプルの総数

まずは簡単な例を作り、手計算と関数の結果を比較してください。

《例》

クラス1:サンプル数15, クラス2:サンプル数15 → ジニ不純度0.500

クラス1:サンプル数15, クラス2:サンプル数15, クラス3:サンプル数15 → ジニ不純度0.667

クラス1:サンプル数18, クラス2:サンプル数12 → ジニ不純度0.480

クラス1:サンプル数30, クラス2:サンプル数0 → ジニ不純度0.000

- 引数の決定
  - クラスとサンプル数を紐付けた辞書型(dict_class_sample)

In [74]:
# 検算1
dict_class_sample = {1: 15, 2: 15}

N = sum(dict_class_sample.values())

tmp_sum = 0
for i in dict_class_sample.keys():
    tmp_sum += (dict_class_sample[i]/N)**2
I_t = 1 - tmp_sum
I_t

0.5

In [75]:
# 検算2
dict_class_sample = {1: 15, 2: 15, 3: 15}

N = sum(dict_class_sample.values())

tmp_sum = 0
for i in dict_class_sample.keys():
    tmp_sum += (dict_class_sample[i]/N)**2
I_t = 1 - tmp_sum
I_t

0.6666666666666667

In [76]:
# 検算3
dict_class_sample = {1: 18, 2: 12}

N = sum(dict_class_sample.values())

tmp_sum = 0
for i in dict_class_sample.keys():
    tmp_sum += (dict_class_sample[i]/N)**2
I_t = 1 - tmp_sum
I_t

0.48

In [151]:
# 検算4
dict_class_sample = {1: 30, 2: 0}

N = sum(dict_class_sample.values())

tmp_sum = 0
for i in dict_class_sample.keys():
    tmp_sum += (dict_class_sample[i]/N)**2
I_t = 1 - tmp_sum
I_t

0.0

- _ginis_diversity_indexを追加

### 【問題2】情報利得を求める関数
次に、ノード間の 情報利得 を計算する関数を作成してください。問題1で作成したジニ不純度 $I(t)$ を計算する関数を呼び出して使います。情報利得$IG$は以下の数式で求まります。うまく分けられている時ほど情報利得は大きくなります。

ここで分岐は2つのみであるため、分岐先を「左側のノード・右側のノード」と呼びます。

$$
IG(p)=I(p)-\frac{N_{left,all}}{N_{p,all}}I(left)-\frac{N_{right,all}}{N_{p,all}}I(right)
$$

$p$ : 親ノードを示すインデックス

$left$ : 左側のノードを示すインデックス

$right$ : 右側のノードを示すインデックス

まずは簡単な例を作り、手計算と関数の結果を比較してください。

《例》

左ノードクラス1:サンプル数10, 左ノードクラス2:サンプル数30, 右ノードクラス1:サンプル数20, 右ノードクラス2:サンプル数5 → 情報利得0.143

- 引数の決定
  - クラスとサンプル数を紐付けた辞書型(左ノード)(dict_class_sample_left)
  - クラスとサンプル数を紐付けた辞書型(右ノード)(dict_class_sample_right)

In [177]:
def _ginis_diversity_index(dict_class_sample):
    """
    ジニ係数を計算する

    Parameters
    ----------
    dict_class_sample : クラスとサンプルのインデックスを紐付けた辞書型
    keyがクラス、valueが各クラスのサンプルのndarray
    """
    N = sum(dict_class_sample.values())
    tmp_sum = 0
    for i in dict_class_sample.keys():
        tmp_sum += (dict_class_sample[i]/N)**2
    I_t = 1 - tmp_sum
    
    return I_t

In [168]:
dict_class_sample_left = {1: 10, 2: 30}
dict_class_sample_right = {1: 20, 2: 5}

p_samples = np.array(list(dict_class_sample_left.values())) + np.array(list(dict_class_sample_right.values()))
dict_class_sample_p = dict(zip(list(dict_class_sample_left.keys()), p_samples)) 

N_p_all = sum(list(dict_class_sample_p.values()))
N_left_all = sum(list(dict_class_sample_left.values()))
N_right_all = sum(list(dict_class_sample_right.values()))

IG = _ginis_diversity_index(dict_class_sample_p) \
     - (N_left_all/N_p_all)*_ginis_diversity_index(dict_class_sample_left) \
     - (N_right_all/N_p_all)*_ginis_diversity_index(dict_class_sample_right)

IG

0.14319526627218937

In [106]:
sum(list(dict_class_sample_left.values()))

40

- _information_gainを追加

### 【問題3】学習
空間の分割を行い、決定木のグラフを生成するコードを作成してください。今は深さ1の決定木なので、分割を1回だけ行います。ここでグラフを生成するとは、1回の分割の際の条件としてどの特徴量がいくつ以上の時とするかを求めるということです。

訓練データに対して全ての組み合わせの分割を行い、その中でノード間の情報利得が最大となる分割をそのノードの分割基準として記録します。

クラスが混ざらない不純度が0のノード、または指定された深さのノードが 葉ノード となります。葉ノードにはクラスを記録しておき、これを推定時に分類するクラスとします。クラスが混ざらない場合はそのままのクラスを記録し、混ざっている場合は多数決により決めます。

《組み合わせの取り方》

全ての組み合わせの取り方は、最も単純には各特徴量の値自体をしきい値にして分割を行う方法があります。片側の端は今回のスクラッチはこの方法で行なってください。

他には中間の値をしきい値にする方法もあり、scikit-learnではこの方法が用いられています。

《補足》

問題2の情報利得を計算する関数はこの問題3で利用する上では、親ノードの不純度 $I(p)$ は固定されるため、左右のノードの不純度の合計を計算するだけでも同じ結果が得られることになります。しかし、ここでは親ノードを考慮した情報利得を計算する実装を行なってください。

アルゴリズム
- 以下を、サンプル数分繰り返す
    - あるサンプルの特徴量をthresholdに設定する
    - thresholdを基準にサンプルを2クラスに分割する
    - IGを求め、list_IGに設定
- max(list_IG)のthresholdを記録しておく

In [216]:
def _information_gain(dict_class_sample_left, dict_class_sample_right):
    """
    情報利得を計算する

    Parameters
    ----------
    dict_class_sample_left : クラスとサンプル数を紐付けた辞書型(左ノード)
    dict_class_sample_right : クラスとサンプル数を紐付けた辞書型(右ノード)
    keyがクラス、valueが各クラスのサンプル数
    """
    p_samples = np.array(list(dict_class_sample_left.values())) + np.array(list(dict_class_sample_right.values()))
    dict_class_sample_p = dict(zip(list(dict_class_sample_left.keys()), p_samples)) 

    N_p_all = sum(list(dict_class_sample_p.values()))
    N_left_all = sum(list(dict_class_sample_left.values()))
    N_right_all = sum(list(dict_class_sample_right.values()))

    rate_left = N_left_all/N_p_all
    rate_right = N_right_all/N_p_all

    if rate_left != 0:
        gini_left = _ginis_diversity_index(dict_class_sample_left)
    else:
        gini_left = 0
    
    if rate_right != 0:
        gini_right = _ginis_diversity_index(dict_class_sample_right)
    else:
        gini_right = 0

    IG = _ginis_diversity_index(dict_class_sample_p) - rate_left*gini_left - rate_right*gini_right

    return IG

In [206]:
X = np.array([[1,4,6],[4,5,6],[10,12,13],[3,4,5],[12,14,15]])
y = np.array([1,1,2,1,2])

In [135]:
np.unique(X[:, 0])

array([ 1,  4,  7, 10, 13])

In [217]:
dict_class_sample_left = {}
dict_class_sample_right = {}

list_info_gain = []

for i in range(X.shape[1]):
    features = np.unique(X[:, i])
    labels = np.unique(y)

    info_gain = []
    for threshold in features:
        sample_left = X[X[:, i] > threshold]
        label_left = y[X[:, i] > threshold]
        sample_right = X[X[:, i] <= threshold]
        label_right = y[X[:, i] <= threshold]
        
        tmp = np.where(label_left == labels[0], True, False)
        dict_class_sample_left[labels[0]] = len(sample_left[tmp])
        tmp = np.where(label_left == labels[1], True, False)        
        dict_class_sample_left[labels[1]] = len(sample_left[tmp])
        
        tmp = np.where(label_right == labels[0], True, False)
        dict_class_sample_right[labels[0]] = len(sample_right[tmp])
        tmp = np.where(label_right == labels[1], True, False)        
        dict_class_sample_right[labels[1]] = len(sample_right[tmp])
        
        print(dict_class_sample_left, dict_class_sample_right)
        
        info_gain.append(_information_gain(dict_class_sample_left, dict_class_sample_right))
     
    list_info_gain.append(info_gain)

list_info_gain

{1: 2, 2: 2} {1: 1, 2: 0}
{1: 1, 2: 2} {1: 2, 2: 0}
{1: 0, 2: 2} {1: 3, 2: 0}
{1: 0, 2: 1} {1: 3, 2: 1}
{1: 0, 2: 0} {1: 3, 2: 2}
{1: 1, 2: 2} {1: 2, 2: 0}
{1: 0, 2: 2} {1: 3, 2: 0}
{1: 0, 2: 1} {1: 3, 2: 1}
{1: 0, 2: 0} {1: 3, 2: 2}
{1: 2, 2: 2} {1: 1, 2: 0}
{1: 0, 2: 2} {1: 3, 2: 0}
{1: 0, 2: 1} {1: 3, 2: 1}
{1: 0, 2: 0} {1: 3, 2: 2}


[[0.07999999999999996, 0.21333333333333332, 0.48, 0.17999999999999994, 0.0],
 [0.21333333333333332, 0.48, 0.17999999999999994, 0.0],
 [0.07999999999999996, 0.48, 0.17999999999999994, 0.0]]

In [208]:
print(X[X[:, 0] > 4])
print(y[X[:, 0] > 4])

[[10 12 13]
 [12 14 15]]
[2 2]


In [201]:
tmp = np.where(y[X[:, 0] > 4] == 1, True, False)
tmp

array([ True, False, False])

In [202]:
X[X[:, 0] > 4][tmp]

array([[7, 8, 9]])

In [195]:
np.logical_not(np.where(y == 1, True, False))

array([False, False, False,  True,  True])

### 【問題4】推定
推定する仕組みを実装してください。ScratchDecesionTreeClassifierDepth1クラスの雛形に含まれるpredictメソッドに書き加えてください。

入力されたデータの値を学習した条件で判定していき、どの葉ノードに到達するかを見ます。葉ノードにはクラスが記録されているので、これが推定値となります。

### 【問題5】学習と推定
機械学習スクラッチ入門のSprintで用意したシンプルデータセット2の2値分類に対してスクラッチ実装の学習と推定を行なってください。

scikit-learnによる実装と比べ、正しく動いているかを確認してください。

AccuracyやPrecision、Recallなどの指標値はscikit-learnを使用してください。

### 【問題6】決定領域の可視化
決定領域を可視化してください。

### 【問題7】（アドバンス課題）深さ2の決定木分類器クラスの作成
深さが2の決定木分類器のクラスScratchDecesionTreeClassifierDepth2を作成してください。

深さ2とは空間の分割を2回行うことを指します。

《ヒント》

各ノードをインスタンスとして扱うと、任意の深さへの拡張が行いやすくなります。

### 【問題8】（アドバンス課題）深さに制限のない決定木分類器クラスの作成
深さに制限のない決定木分類器のクラスScratchDecesionTreeClassifierDepthInfを作成してください。

任意の深さを指定できるようにするとともに、指定しない場合は全ての葉ノードがジニ不純度0となるまで続けられるようにもしてください。